# Imports

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import networkx as nx
import re
from matplotlib import pyplot as plt
import time
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict

# Load statics

In [ ]:
# Load stored course-site HTMLs
with open('valid_courses.json') as f:
    valid_courses = json.load(f)

# Load department color mapping
with open('department_colors.json') as f:
    department_colors = json.load(f)

# Load department names
with open('department_names.json') as f:
    department_names = json.load(f)

# Create graph

## Add nodes

In [ ]:
# Initialize directd graph
G = nx.DiGraph()

# Go through each course
for course_num in valid_courses:
    department = course_num[:2] 
    G.add_node(course_num,
               course_num=course_num,
               page=valid_courses[course_num],
               department=department, 
               color=department_colors[department],
               department_name=department_names[department])

## Add edges

In [ ]:
for course_num in valid_courses:

    ### Initialize BeuatifulSoup object
    page = G.nodes[course_num]['page']
    soup = BeautifulSoup(page, 'html.parser')

    ### Find <tr> that contains "Academic prerequisites"
    search_string = "Academic prerequisites"
    
    # Find respective label
    label = soup.find('label', string=re.compile(search_string))
    if label is None:
        continue
    
    # Get element that contains the label + prerequisites
    parent = label.find_parent().find_parent()
    
    # Get second <td>
    prerequisite = parent.find_all('td')[1].text

    # Remove whitespace and breaks (\n and \r)
    prerequisite  = prerequisite.replace('\r', ' ')
    prerequisite  = prerequisite.replace('\n ', '')

    # Extract 5 digit numbers (any course number)
    prerequisites = set(re.findall(r'\d{5}', prerequisite))
    
    # if course exists in graph, add edge
    for prerequisite in prerequisites:
        if prerequisite in G.nodes:
            if prerequisite == course_num: 
                # Handle self-loops
                pass
            else:
                G.add_edge(prerequisite, course_num)

## Add text attributes

In [ ]:
for node in G.nodes:
    
    ### Initialize BeuatifulSoup object
    page = G.nodes[node]['page']
    soup = BeautifulSoup(page, 'html.parser')
    
    ### Add title to node
    title = soup.title.text
    cleaned = ' '.join(title.strip().split())[6:]
    G.nodes[node]['course_title'] = cleaned
    
    ### Add course text to node (General course objectives + Learning objectives + Content)
    div = soup.find('div', string=re.compile("General course objectives")).parent(string=True)
    
    remove_indeces = []
    for d, text in enumerate(div):
        if text in ["General course objectives", "Learning objectives", "Content", "Last updated", "\r\nA student who has met the objectives of the course will be able to:\r\n\r\n"]:
            remove_indeces.append(d)
    
    new_div = [div[i] for i in range(len(div)) if i not in remove_indeces]
    text = ' '.join(new_div[:-1]).replace('\r', ' ').replace('\n', '')
    cleaned = ' '.join(text.strip().split())
    G.nodes[node]['course_text'] = cleaned  
    G.nodes[node]['text_size'] = len(cleaned) 
    G.nodes[node]['word_count'] = len(cleaned.split())

In [ ]:
# plot distribution of text sizes
text_sizes = [G.nodes[node]['word_count'] for node in G.nodes]
plt.hist(text_sizes, bins=50, color='C2', alpha=0.7)
plt.title('Distribution of text sizes\n(Number of words)')
plt.xlabel('Word count')
plt.ylabel('Frequency')
plt.show()

# Get evaluations and grades

In [ ]:
# Evaluations
with open('eval_scores.json') as f:
    eval_scores = json.load(f)

# Grades
with open('grade_scores.json') as f:
    grade_scores = json.load(f)

In [ ]:
# Apply evaluation and grade attributes
for course_num in grade_scores:
    G.nodes[course_num]['eval_scores'] = eval_scores[course_num]
    G.nodes[course_num]['grade_scores'] = grade_scores[course_num]
    
# Remove nodes from G that do not have evaluation scores
G = G.subgraph([node for node in G.nodes if 'eval_scores' in G.nodes[node]])

In [ ]:
G.nodes['02805']

# Degree distribution

In [ ]:
# Calculate in- and out-degrees. Show histogram
in_degrees = [G.in_degree(n) for n in G.nodes]
out_degrees = [G.out_degree(n) for n in G.nodes]

fig, axs = plt.subplots(1, 2, figsize=(18, 5))
axs[0].set_ylim(0, 900)
axs[0].hist(out_degrees, bins=range(min(out_degrees), max(out_degrees) + 2), color='C4', alpha=0.7, edgecolor='grey')
axs[0].set_xlabel('Is a prerequisite for X courses')
axs[0].set_ylabel('Number of courses')
axs[0].set_title('Out-degree distribution')
axs[0].xaxis.set_major_locator(plt.MaxNLocator(integer=True))
axs[0].xaxis.set_major_locator(plt.MultipleLocator(5))

axs[1].set_ylim(0, 900)
axs[1].hist(in_degrees, bins=range(min(in_degrees), max(in_degrees) + 2), color='C1', alpha=0.7, edgecolor='grey')

axs[1].set_xlabel('Has X possible prerequisites')
axs[1].set_ylabel('Number of courses')
axs[1].set_title('In-degree distribution')
axs[1].xaxis.set_major_locator(plt.MaxNLocator(integer=True))
axs[1].xaxis.set_major_locator(plt.MultipleLocator(1))

plt.show()

# Plotting

## Test partitionings

In [ ]:
import networkx as nx
from collections import Counter, defaultdict
import random

def overlapping_label_propagation(graph, max_size, max_iterations=100):
    """
    Overlapping Label Propagation Algorithm with size constraints.

    Parameters:
        graph (nx.Graph): The input graph.
        max_size (int): The maximum allowable size for a community.
        max_iterations (int): Maximum number of iterations to run the algorithm.

    Returns:
        dict: A dictionary where keys are community indices (starting from 0) and values are lists of nodes.
    """
    # Initialize each node with its own unique label as a set
    labels = {node: {node} for node in graph.nodes}
    
    for iteration in range(max_iterations):
        changes = False
        
        # Randomize node order to reduce bias
        nodes = list(graph.nodes)
        random.shuffle(nodes)
        
        for node in nodes:
            # Collect all labels from neighbors
            neighbor_labels = []
            for neighbor in graph.neighbors(node):
                neighbor_labels.extend(labels[neighbor])  # Collect labels of neighbors
            
            # Count the frequency of each label in the neighborhood
            label_counts = Counter(neighbor_labels)
            
            # Sort labels by frequency (descending) and resolve ties randomly
            sorted_labels = sorted(label_counts.items(), key=lambda x: (-x[1], random.random()))
            
            # Add labels while respecting size constraints
            current_labels = labels[node]
            for label, _ in sorted_labels:
                community_size = sum(1 for n in graph.nodes if label in labels[n])
                if community_size < max_size and label not in current_labels:
                    current_labels.add(label)
                    changes = True
            
        # Stop if no labels were changed
        if not changes:
            break
    
    # Create a community dictionary with sequential indices
    community_dict = defaultdict(list)
    label_to_index = {}  # Map original labels to indices
    current_index = 0
    
    for node, node_labels in labels.items():
        for label in node_labels:
            if label not in label_to_index:
                label_to_index[label] = current_index
                current_index += 1
            community_dict[label_to_index[label]].append(node)
    
    return dict(community_dict)



In [126]:
def heuristic_partitioning(graph):
    '''
    For each node, add a community that consists of:
    1. The node itself
    2. Predecessors and the predecessors of predecessors
    3. Successors and the successors of successors 
    '''
    communities = defaultdict(list)  # Store communities indexed by node
    
    for node in graph.nodes:
        # Initialize the community with the node itself
        community = set([node])
        
        # Add predecessors and their predecessors
        for pred in graph.predecessors(node):
            community.add(pred)
            community.update(graph.predecessors(pred))
        
        # Add successors and their successors
        for succ in graph.successors(node):
            community.add(succ)
            community.update(graph.successors(succ))
        
        # Convert the community set to a list and assign it
        communities[node] = list(community)
    
    return dict(communities)


In [ ]:
# Method 1 - Ordinary label propagation
from networkx.algorithms.community import label_propagation_communities
H = G.to_undirected()
communities = label_propagation_communities(H)

In [ ]:
# Method 2 - Overlapping label propagation with size constraints
H = G.to_undirected()
communities = overlapping_label_propagation(H, 6)

In [ ]:
# Method 3 - Overlapping label propagation with size constraints and degree filtering
H = G.to_undirected()
for node in G.nodes:
    if G.out_degree(node) > 10:
        H.remove_node(node)  
communities = overlapping_label_propagation(H, 6)

In [127]:
# Method 4 - Heuristic ± 2 (directed graph with constraints)
H = G.copy()
communities = heuristic_partitioning(H)

In [128]:
communities

{'01001': ['26231',
  '46115',
  '02582',
  '02417',
  '10325',
  '02323',
  '02405',
  '34231',
  '26236',
  '01405',
  '02141',
  '22052',
  '02182',
  '02443',
  '02504',
  '34220',
  '41111',
  '01001',
  '41342',
  '02431',
  '02407',
  '02509',
  '02506',
  '02421',
  '41822',
  '12104',
  '02960',
  '26020',
  '47310',
  '02586',
  '02464',
  '02403',
  '02249',
  '01527',
  '10240',
  '22283',
  '01004',
  '42417',
  '02460',
  '42186',
  '02502',
  '26261',
  '41501',
  '41129',
  '01125',
  '41275',
  '02245',
  '46000',
  '02953',
  '02445',
  '26211',
  '02291',
  '01037',
  '02456',
  '02562',
  '26263',
  '01415',
  '02263',
  '02526',
  '02450',
  '46211',
  '41317',
  '42588',
  '02501',
  '41320',
  '01715',
  '01020',
  '02411',
  '41315',
  '10260',
  '30405',
  '46040',
  '42586',
  '22525',
  '41216',
  '02601',
  '41226',
  '10331',
  '41969',
  '02492',
  '01018',
  '02269',
  '02471',
  '10350',
  '41524',
  '42380',
  '46110',
  '46765',
  '02238',
  '46500',
 

In [ ]:
# plot distribution of value sizes
value_sizes = [len(communities[c]) for c in communities]
plt.hist(value_sizes, bins=range(min(value_sizes), max(value_sizes) + 2), color='C2', alpha=0.7)
plt.title('Distribution of community sizes\n(Number of courses)')
plt.xlabel('Number of courses')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# plot subgraph of G belong to community X
index = 50
subgraph = G.subgraph(communities[index])
pos = nx.circular_layout(subgraph)
plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(subgraph, pos, node_size=100, node_color='C0')
nx.draw_networkx_edges(subgraph, pos, alpha=0.5)
# labels = course_title
nx.draw_networkx_labels(subgraph, pos, labels={node: '\n'+G.nodes[node]['course_title'] for node in subgraph.nodes}, font_size=8)

plt.title(f'Community {index}')
plt.show()

### Department only

In [ ]:
# plot subgraph of G, where department is X
department = '02'
subgraph = G.subgraph([node for node in G.nodes if G.nodes[node]['department'] == department])
pos = nx.circular_layout(subgraph)
plt.figure(figsize=(10, 10))
nx.draw_networkx_nodes(subgraph, pos, node_size=100, node_color=department_colors[department])
nx.draw_networkx_edges(subgraph, pos, alpha=0.5)
# labels = course_title
nx.draw_networkx_labels(subgraph, pos, labels={node: G.nodes[node]['course_title'] for node in subgraph.nodes}, font_size=8)
plt.title(department_names[department])
plt.show()

# Summarize communities

In [ ]:
G.nodes['01001']

In [ ]:
class CommunitiesSummary():
    def __init__(self, graph, communities):
        self.graph = graph
        self.communities = communities
    
    # Number of communities
    def __len__(self):
        return len(self.communities)
    
    # Average grade
    def avg_grade(self, community_index):
        grades = []
        for course in self.communities[community_index]:
            grade = self.graph.nodes[course]['grade_scores'][2]
            
            # Only add if the course uses the 7-scale grading system
            if grade is not None:
                grades.append(grade)
        
        # Return None if no grades were found
        if len(grades) == 0:
            return None
        
        # Return the average grade
        return sum(grades) / len(grades)
    
    # Average pass rate
    def avg_pass_rate(self, community_index):
        pass_rates = []
        for course in self.communities[community_index]:
            grade_scores = self.graph.nodes[course]['grade_scores']
            pass_rates.append(grade_scores[1] / grade_scores[0])
       
        # Return average pass rate 
        return sum(pass_rates) / len(pass_rates)
    
    # Average evaluation score
    def avg_evals(self, community_index):
        
        all_evals = []
        for i in range(6):
            evals = []
            for course in self.communities[community_index]:
                eval_score = self.graph.nodes[course]['eval_scores'][i]
                evals.append(eval_score)
            all_evals.append(sum(evals)/len(evals))
        
        return all_evals
    
    # Get single community stats
    def __getitem__(self, index):
        community = self.communities[index]
        return {
            'index': index,
            'size': len(community),
            'courses': community,
            'courses_titles': [self.graph.nodes[course]['course_title'] for course in community],
            'avg_grade': self.avg_grade(index),
            'avg_pass_rate': self.avg_pass_rate(index),
            'avg_evals': self.avg_evals(index),
            'all_text': ' '.join([self.graph.nodes[course]['course_text'] for course in community])
        }
        
    # Get all communities that includes course
    def get_communities(self, course):
        return [index for index, community in self.communities.items() if course in community]
   
        
COM = CommunitiesSummary(G, communities)

In [ ]:
COM[20]

In [ ]:
aml = COM.get_communities('02460')

for i in aml:
    display(COM[i])
    print()